In [11]:
# Imports

import matplotlib
import numpy as np
from enum import Enum
from collections import deque
from queue import PriorityQueue
from queue import Queue
from copy import copy, deepcopy
import sys
import random

In [12]:
# Coordinate Class
# Source: http://pythonfiddle.com/coordinate-class/

class Coordinate(object):
    
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def getX(self):
        return self.x
    def getY(self):
        return self.y
    def getCoordinates(self):
        return self.x, self.y

    def __str__(self):
        return '<' + str(self.getX()) + ',' + str(self.getY()) + '>'  
    def __unicode__(self):
        return '<' + str(self.getX()) + ',' + str(self.getY()) + '>'
    def __repr__(self):
        return '<' + str(self.getX()) + ',' + str(self.getY()) + '>'
    
    def __eq__(self, other):
        return self.y == other.y and self.x == other.x
    
    #https://stackoverflow.com/questions/9135759/java-hashcode-for-a-point-class
    def __hash__(self):
        result = self.x;
        result = 31 * result + self.y;
        return result;

In [13]:
# RoomElement Enum

class RoomElement(Enum):
    EMPTY = 0
    WALL = 1
    DOOR = 2
    OBSTACLE = 3
    EXIT = 4

In [14]:
# Room Class

class Room():
    
    def __init__(self):
        self.map = np.array([[]])
        
    def __init__(self, m):
        self.map = np.array(m)
        
    def get_element_coordinates_of_map(self, roomElement):
        # roomElement = RoomElement.EMPTY
        # find all coordinates in map that have value 0
        spots = []
        y = 0
        for row in self.map:
            x = 0
            for value in row:
                if value == roomElement.value:
                    coord = Coordinate(y, x)
                    spots.append(coord)
                x += 1   
            y += 1
        return spots
    
    def print_current_map_spot(self, c: Coordinate):
        mapCopy = deepcopy(self.map)
        mapCopy[c.x][c.y] = 9999
        return mapCopy
    
    def print_current_map_spots(self, mapSpots):
        mapCopy = deepcopy(self.map)
        for spot in mapSpots:
            mapCopy[spot.x][spot.y] = 9999
        return mapCopy
    
    def paths_to_a_goal(self, start: Coordinate, end: Coordinate):
        stack = Queue()
        visited = set()
        possiblePlans = []
        actions = []
        
        stack.put((start, actions))
        
        while not stack.empty():
            tupleObject = stack.get()
            coord = tupleObject[0]
            if coord.getCoordinates() in visited:
                continue
            
            actionListSoFar = tupleObject[1]
            
            visited.add(coord);
            if coord == end:
                possiblePlans.append(actionListSoFar)
                continue
            
            successors = []
            successors.append(Coordinate(coord.x - 1, coord.y))
            successors.append(Coordinate(coord.x + 1, coord.y))
            successors.append(Coordinate(coord.x, coord.y - 1))
            successors.append(Coordinate(coord.x, coord.y + 1))
            
            for c in successors:
                if c.x < 0 or c.y < 0 or c.x >= self.map.shape[0] or c.y >= self.map.shape[1]:
                    continue
                if self.map[c.x][c.y] == RoomElement.WALL.value:
                    continue
                
                if (not c in visited):
                    newActions = actionListSoFar + [c]
                    stack.put((c, newActions))
        
        return possiblePlans
    
    def best_path(self, paths):
        bestDistance = sys.maxsize
        bestPath = []
        
        for path in paths:
            unitsMoved = len(path)
            if (unitsMoved < bestDistance):
                bestDistance = unitsMoved
                bestPath = path
        
        return bestPath

In [15]:
# Person Class
class Person():
    
    def __init__(self):
        self.currentCoordinate = Coordinate(0, 0)
        self.path = []
        self.index = 0
        self.exited = False
        
    def __init__(self, c, p):
        self.currentCoordinate = c
        self.index = 0
        self.path = p
        self.exited = False
        
    def hasExited(self):
        if self.index >= len(self.path):
            self.exited = False
        return self.exited
    
    def get_current_coordinate(self):
        return self.currentCoordinate
        
    def next_spot_in_path(self):
        if self.hasExited():
            return None
        if (self.index + 1 >= len(self.path)):
            return None
        nextSpot = self.path[self.index + 1]
        return nextSpot
    
    def move_to_next_spot_in_path(self):
        self.update_index(self.index + 1)
        self.currentCoordinate = self.path[self.index]
        if self.index >= len(self.path) - 1:
            self.exited = True
        if self.path[-1] == self.currentCoordinate:
            self.exited = True
        
    def update_index(self, i):
        self.index = i
        
    def __str__(self):
        return 'Person: @' + str(self.currentCoordinate) + ', Path: ' + str(self.path) + ', Exited? ' + str(self.exited)  
    def __unicode__(self):
        return 'Person: @' + str(self.currentCoordinate) + ', Path: ' + str(self.path) + ', Exited? ' + str(self.exited)  
    def __repr__(self):
        return 'Person: @' + str(self.currentCoordinate) + ', Path: ' + str(self.path) + ', Exited? ' + str(self.exited)  

In [16]:
# Helper Methods
def random_elements_of_list(list, N, K):
    newList = []
    indexes = random.sample(range(N), K)
    for index in indexes:
        newList.append(list[index])
    return newList

In [17]:
# EvacuationSimulation Class
class EvacuationSimulation():
    
    def __init__(self):
        self.room = Room()
        self.peopleCoordinates = []
        
    def __init__(self, r, p):
        self.room = r
        self.peopleCoordinates = p
        
    def get_path_based_on_current_coordinate(self, coordinate, room):
        paths = []
        exits = room.get_element_coordinates_of_map(RoomElement.EXIT)
        for exit in exits:
            somePaths = self.room.paths_to_a_goal(coordinate, exit)
            paths.extend(somePaths)
        bestPath = room.best_path(paths)
        return bestPath
    
    def everyone_has_exited(self, personList):
        for person in personList:
            if not person.hasExited():
                print('Not exited: ' + str(person))
                return False
        return True

    def simulate(self):
        personList = []
        for coordinate in self.peopleCoordinates:
            personPath = self.get_path_based_on_current_coordinate(coordinate, self.room)
            newPerson = Person(coordinate, personPath)
            personList.append(newPerson)
        
        print('Initialize simulation')
        print(self.room.print_current_map_spots(self.peopleCoordinates))
        print(personList)
        
        
        time = 0
        
        # Start Loop
        
        while not self.everyone_has_exited(personList):
            locationDict = {}
            for person in personList:
                if (person.next_spot_in_path() is None):
                    continue
                nextPlaceInPath = person.next_spot_in_path()
                print("Next spot: " + str(nextPlaceInPath))
                
                if nextPlaceInPath in locationDict and locationDict[nextPlaceInPath] is not None and len(locationDict[nextPlaceInPath]) > 1:
                    oldList = locationDict[nextPlaceInPath]
                    newList = oldList.append(person)
                    locationDict[nextPlaceInPath] = newList
                else:
                    locationDict[nextPlaceInPath] = [person]
                    
            
            print(locationDict)
            
            for key in locationDict.keys():
                print(key)
                print(locationDict[key])
                if locationDict[key] is None:
                    continue
                
                if len(locationDict[key]) == 1:
                    # person can move on
                    personToMove = locationDict[key][0]
                    personToMove.move_to_next_spot_in_path()
                elif len(locationDict[key]) > 1:
                    # one random person moves forward
                    randomIndex = random.sample(range(len(locationDict[key])), 1)
                    persontoMove = locationDict[key][randomIndex]
                    personToMove.move_to_next_spot_in_path()
            time = time + 1
            print('Time: ' + str(time))
            
            coordinates = []
            for person in personList:
                c = person.get_current_coordinate()
                coordinates.append(c)
            print('Coordinates')
            print(coordinates)
            print(self.room.print_current_map_spots(coordinates))
            
            
        # End Loop
        
        print('Done with simulation')
                    
        
        ''' 
        ^Goal: Everyone tried to move forward one place in their path list. 
            If multiple people try to go to same spot, one of the people is randomly chosen 
            to move forward by collision logistics. The other will stay in their spot.
            This happens until everyone has reached an exit. At that point, we take stock
            of how much time passed (where every person moving one unit is considered a unit of time).
        '''
        

In [18]:
# ROOM 1
# One exit in a walled empty room.
room_1_map = [
    [1, 1, 1, 1, 1, 1, 1, 1, 4, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]
room_1 = Room(room_1_map)
room_1_exits = room_1.get_element_coordinates_of_map(RoomElement.EXIT)

paths = room_1.paths_to_a_goal(Coordinate(3, 7), room_1_exits[0])

print('Paths: ' + str(paths))
print('Best Path: ' + str(room_1.best_path(paths)))


room_1_empty_spots = room_1.get_element_coordinates_of_map(RoomElement.EMPTY)
room_1_empty_spots_subset = random_elements_of_list(room_1_empty_spots, len(room_1_empty_spots), int(len(room_1_empty_spots)/10))
print('Character Spots: ' + str(room_1_empty_spots_subset))

simulation1 = EvacuationSimulation(room_1, room_1_empty_spots_subset)
simulation1.simulate()

Paths: [[<2,7>, <1,7>, <1,8>, <0,8>], [<2,7>, <2,8>, <1,8>, <0,8>], [<3,8>, <2,8>, <1,8>, <0,8>]]
Best Path: [<2,7>, <1,7>, <1,8>, <0,8>]
Character Spots: [<6,2>, <8,8>, <3,6>, <4,5>, <6,7>, <1,6>]
Initialize simulation
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0 9999    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0 9999    0    0    1]
 [   1    0    0    0    0 9999    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0 9999    0    0    0    0 9999    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0 9999    1]
 [   1    1    1    1    1    1    1    1    1    1]]
[Person: @<6,2>, Path: [<5,2>, <4,2>, <3,2>, <2,2>, <1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False, Person: @<8,8>, Path: [<7,8>, <6,8>, <5,8>, <4,8>, <3,8>, <2,8>, <1,8>, <0,8>], Exited? False, Person: @<3,6>, Path: [<2,6>, 

In [19]:
# ROOM 2
# Same as room 1 but the walking path is a square.
room_2_map = [
    [1, 1, 1, 1, 1, 1, 1, 1, 4, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]

room_2 = Room(room_2_map)
room_2_exits = room_2.get_element_coordinates_of_map(RoomElement.EXIT)
# print(room_2_exits)

# print(room_2.print_current_map_spot(Coordinate(1, 1)))

paths = room_2.paths_to_a_goal(Coordinate(1, 1), room_2_exits[0])

# print(paths)
# print(room_2.best_path(paths))

In [20]:
# ROOM 3
# Same as Room 1 but with two exits.
room_3_map = [
    [1, 1, 1, 1, 1, 1, 1, 1, 4, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 4, 1, 1, 1, 1, 1, 1, 1, 1]
]

room_3 = Room(room_3_map)
room_3_exits = room_3.get_element_coordinates_of_map(RoomElement.EXIT)
# print(room_3_exits)

# print(room_3.print_current_map_spot(Coordinate(3, 7)))
paths = room_3.paths_to_a_goal(Coordinate(3, 7), room_3_exits[0])
# print(room_3.best_path(paths))

# print(room_3.print_current_map_spot(Coordinate(1, 1)))
paths = room_3.paths_to_a_goal(Coordinate(1, 1), room_3_exits[1])
# print(room_3.best_path(paths))

In [22]:
# Checkpoint Notes
'''
1. Implement a hazard. - Rohan
Fire can expand at random rates (stochastic).

2. Different rates of people. - Rohan
Define that? Each person has different average walking speed and standard deviation.
Subclasses for People, each with statistics. And different behavior for collisions.

3. Randomized map generator - Rishab & Dheeraj

4. Best optimal room evaluation (evaluation score) - Rishab & Dheeraj
    How to do this? Put notes here.

''';

In [ ]:
# Randomize Map Method 1
'''
Randomly generate NxM array (25 > N > 5, 25 > M > 5) 
where the room is bounded by walls and all of the inner space is empty.
However, one space in the wall that isn't a corner will be a door.
'''

In [ ]:
# Randomize Map Method 2
'''
Randomly generate NxM array (25 > N > 5, 25 > M > 5) 
where the room is bounded by walls and all of the inner space is empty.
However, multiple doors will exist in the in the wall (6 > D > 1) (none of which isn't a corner).
'''

In [ ]:
# Randomize Map Method 3
'''
Randomly generate NxM array (25 > N > 5, 25 > M > 5) 
where the room is bounded by walls and all of the inner space is empty.
However, one doors will exist in the in the wall that is multiple units long (consecutive D's in the wall),
and none of the D's can exist in any corner.
'''

In [ ]:
# Randomize Map Method 4
'''
Randomly generate NxM array (25 > N > 5, 25 > M > 5) 
where the room is bounded by walls and all of the inner space is empty.
However, random columns of walls exist that jut out from the walls. 
A person would still need to be able to get to an exist.
'''
# Example:
'''
11111
10001
10001
10101
10101
11111
'''